# Yandex Algorithm 2018 ML Track

## простой baseline на основе tfidf 
при подкрутке параметров может давать более 84000-85500

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('datasets/yandex/train.tsv', sep='\t', quoting=3, header=None)

In [3]:
test = pd.read_csv('datasets/yandex/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None)

In [4]:
test.shape

(9968, 6)

In [5]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [6]:
train.sample(3)

,0,1,2,3,4,5,6,7
53391,153621250860507,"ок , итак . . .",привет .,у меня кое - что есть .,2,окей .,neutral,0.777938
34215,98333000555177,,"энди , я не знаю , останешься ли ты копом , но...","да уж , какое - то время кишки наружу торчали .",1,скорее всего ветер принес их сюда .,bad,0.865455
96141,277564398244016,,,"я рад равно последним , как и первым .",2,ушел в минус ?,bad,0.966602


In [7]:
test.sample(3)

,0,1,2,3,4,5
8226,229638918112133,,,очень холодно .,2,"я думала , в пустыне всегда жарко ."
4490,128083742551422,,,"спасибо , мам .",5,я так тобой горда .
2446,66429986634329,или внутрь пойду .,"я захожу , увидимся там .","если нет , тогда может позвоню тебе или еще что .",4,"там мой номер , на случай , если ты захочешь м..."


In [8]:
# расставляем значения по формуле хорошесть * confidence
# Good (2)
# Neutral (1)
# Bad (0)

In [9]:
def rank2num(st):
    if st == 'good':
        return 2
    else:
        if st == 'neutral':
            return 1
        else:
            return 0

In [10]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,2,1.750703
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,0,0.000000
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,2,1.965061
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,2,1.676107


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
content = train[1].append(train[2]).append(train[3]).append(train[5]).append(test[1]).append(test[2]).append(test[3]).append(test[5])

In [13]:
%%time
tfidf = TfidfVectorizer().fit(content)

CPU times: user 4.06 s, sys: 44.9 ms, total: 4.1 s
Wall time: 4.23 s


In [14]:
t1 = tfidf.transform(train[1])
t2 = tfidf.transform(train[2])
t3 = tfidf.transform(train[3])
t5 = tfidf.transform(train[5])

te1 = tfidf.transform(test[1])
te2 = tfidf.transform(test[2])
te3 = tfidf.transform(test[3])
te5 = tfidf.transform(test[5])

In [15]:
%%time
import scipy
from scipy.sparse import coo_matrix

X_train = scipy.sparse.hstack([t1, t2, t3, t5]).tocsr()
X_test  = scipy.sparse.hstack([te1, te2, te3, te5]).tocsr()

CPU times: user 401 ms, sys: 28 ms, total: 429 ms
Wall time: 429 ms


In [16]:
X_train.shape, X_test.shape

((97533, 172024), (9968, 172024))

In [17]:
y_train = train['target']

In [18]:
train_part_size = int(0.75 * train['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [19]:
X_train_part.shape

(73149, 172024)

In [20]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

reg = Ridge()

reg.fit(X_train_part, y_train_part)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
reg_pred = reg.predict(X_valid)

In [22]:
# я знаю, что метрика ниже - неправильная, но на практике её снижение даёт уверенный рост на лидерборде

In [23]:
%%time
from sklearn.metrics import mean_absolute_error

valid_mae = mean_absolute_error(y_valid, reg_pred)
print(valid_mae)

0.731022440533
CPU times: user 2.66 ms, sys: 69 µs, total: 2.73 ms
Wall time: 2.88 ms


In [24]:
%%time
reg.fit(X_train, y_train)

CPU times: user 7.82 s, sys: 162 µs, total: 7.82 s
Wall time: 7.87 s


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [25]:
y_test = reg.predict(X_test)

In [26]:
sub = pd.DataFrame()

In [27]:
test.sample(3)

,0,1,2,3,4,5
2944,83430561070243,,"маленькую сумку положите , пожалуйста . . . на...","на заднее , слышите ?",2,"нет , я не могу принести автомобиль сюда !"
8527,239095428377733,,"что ж , ты добился своего .","настоящая правда в том , что у меня действител...",0,"иногда , они теряют сами себя ."
2106,57280934855459,,,новый рекорд студента по обмену !,5,повышение ?


In [28]:
sub['context_id'] = test[0]
sub['reply_id'] = test[4]
sub['rank'] = - y_test

In [29]:
sub.head()

,context_id,reply_id,rank
0,138920940977,0,-0.919516
1,138920940977,1,-1.212823
2,138920940977,2,-0.998551
3,138920940977,3,-1.269788
4,138920940977,4,-1.332044


In [30]:
submission = sub.sort_values(by=['context_id', 'rank'])

In [31]:
del submission['rank']

In [32]:
submission.head()

,context_id,reply_id
4,138920940977,4
3,138920940977,3
1,138920940977,1
5,138920940977,5
2,138920940977,2


In [33]:
test.shape, sub.shape

((9968, 6), (9968, 3))

In [34]:
submission.to_csv('yandex-ml-ridge-baseline-very-simple.tsv',header=None, index=False, sep=' ')